In [194]:
#Maria Williams  - Nov, 2021
#scraper to pull list of novel adaptations from IMDB

In [1]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import requests
import re

In [2]:
#First Pass:
#1) feature films, 2) tagged 'based on novel', 3) release from years 2000 to 2021: 3,516 titles - 71 pages
#baseurl = 'https://www.imdb.com/search/keyword/?keywords=based-on-novel&ref_=kw_nxt&mode=detail&page={}&title_type=movie&release_date=2000%2C2021&sort=year,desc'

#Second Pass:
#1) feature films, 2) tagged 'based on novel', 3) release from years 2000 to 2021: 6,262 titles - 126 pages
baseurl = 'https://www.imdb.com/search/keyword/?keywords=based-on-novel&ref_=kw_ref_yr&mode=detail&page={}&title_type=movie&release_date=1970%2C2000&sort=year,desc'

#initialize dataframe
popcorn = pd.DataFrame(columns = ['Link','Title', 'Release_Date', 'ViewRating', 'Runtime', 'Genres', 'Description'])


In [3]:
def makestring(bs4):
    if bs4 is None:
        return ' '
    else:
        return bs4.string
    
def GetPage(baseurl,num):
    
    pop = pd.DataFrame(columns = ['Link','Title', 'Release_Date', 'ViewRating', 'Runtime', 'Genres', 'Description'])
    
    page = requests.get(baseurl.format(num))
    soup = BeautifulSoup(page.content, 'html.parser')
    main = soup.find_all(attrs={'class':'lister-item mode-detail'})
    #print(len(main))

    for p in range(len(main)):
        #get title
        link = main[p].a['href']
        title = main[p].h3.a.string
        #get description
        desc = main[p].find_all('p')
        desc = desc[1].string
        #get a bunch of other things that may or may not be there  
        year = makestring(main[p].find(attrs={'class':'lister-item-year text-muted unbold'}))
        rating = makestring(main[p].find(attrs={'class':'certificate'}))
        length = makestring(main[p].find(attrs={'class':'runtime'}))
        genre = makestring(main[p].find(attrs={'class':'genre'}))
        #get number of stars - nvm there are always four shown
        #block = main[p].find_all('p')
        #bock = str(block[2])
        #stars = what[what.find("Stars:"):].count('a href')
        #print(title,year,rating,length,genre,desc)
        gotit = [link,title,year,rating,length,genre,desc]
        pop.loc[len(pop)] = gotit

    #print(pop.info())
    return pop

In [4]:
#each page has 50 entries
#First pass: loop to get a 71 pages (full results for seach criteria)
#Second pass: loop to get 126 pages

for page in range(126):
    popper = GetPage(baseurl,page+1)
    popcorn = popcorn.append(popper)

popcorn['Genres'] = popcorn['Genres'].str.strip()
popcorn['Description'] = popcorn['Description'].str.strip()

In [5]:
print(popcorn.info())
print(popcorn.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6262 entries, 0 to 11
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Link          6262 non-null   object
 1   Title         6262 non-null   object
 2   Release_Date  6262 non-null   object
 3   ViewRating    6262 non-null   object
 4   Runtime       6262 non-null   object
 5   Genres        6262 non-null   object
 6   Description   3538 non-null   object
dtypes: object(7)
memory usage: 391.4+ KB
None
                Link                            Title Release_Date ViewRating  \
0  /title/tt0075959/  Jin yu liang yuan hong lou meng       (2007)              
1  /title/tt0144084/                  American Psycho       (2000)          R   
2  /title/tt0180093/              Requiem for a Dream       (2000)          R   
3  /title/tt0266308/                    Battle Royale       (2000)  Not Rated   
4  /title/tt0190590/       O Brother, Where Art Thou?       (2000)

In [6]:
popcorn.to_csv('FilmList2.csv', index = False)